In [ ]:
## Script for the analysis of TCR receptor sequencing
# By Louise Baldwin
# takes T cell object with TCR data as input
# produces figures, plus a head of subsetted adata objects


In [ ]:
# set up
import os
import sys
import scanpy as sc
import scirpy as ir
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import altair_saver as als

from matplotlib.pyplot import rc_context
from altair_saver import save

os.chdir("/share/ScratchGeneral/loubal/projects/MSC/mouse-single-cell/")
in_file =("data/processed/Tcells_withTCR.h5ad")
results_file=("data/processed/Tcells_TCR_analysed.h5ad")
tabdir = ("outs/Tcells_withTCR/tables/")
figdir = ("outs/Tcells_withTCR/figures/")
os.makedirs(tabdir, exist_ok=True)
os.makedirs(figdir, exist_ok=True)

sc.settings.verbosity = 3             
sc.logging.print_header()
sc.settings.set_figure_params(dpi=300, facecolor='white')
sc.settings.figdir=figdir

In [ ]:
adata=sc.read(in_file)

In [ ]:
adata

In [ ]:
# subset just IT and IgG
l=["Control IgG", "PD1+CTLA4"]
adata=adata[adata.obs.Treatment.isin(l)]

In [ ]:
sc.pl.umap(adata, color="Tcell_type", frameon=False, save='.pdf')

In [ ]:
sc.pl.umap(adata, color='Tissue', frameon=False, groups="Primary tumour", save='Tcells_PT.png')
sc.pl.umap(adata, color='Tissue', frameon=False, groups="Lymph node", save="Tcells_LN.png")

In [ ]:
with open(tabdir+"Tcell_LN_number.txt", "w") as text_file:
    print(
        "Number of Tcells from lymph nodes: {:.2f}".format(
            np.sum(
                adata.obs["Tissue"].isin(
                    ["Lymph node"]
                )
            )
#            / adata.n_obs
        ), file=text_file)

with open(tabdir+"Tcell_LN_proportion.txt", "w") as text_file:
    print(
        "Proportion of Tcells from lymph nodes: {:.2f}".format(
            np.sum(
                adata.obs["Tissue"].isin(
                    ["Lymph node"]
                )
            )
            / adata.n_obs
        ), file=text_file)

with open(tabdir+"Tcell_PT_number.txt", "w") as text_file:
    print(
        "Number of Tcells from primary tumours: {:.2f}".format(
            np.sum(
                adata.obs["Tissue"].isin(
                    ["Primary tumour"]
                )
            )
#            / adata.n_obs
        ), file=text_file)


with open(tabdir+"Tcell_PT_proportion.txt", "w") as text_file:
    print(
        "Proportion of Tcells from primary tumour: {:.2f}".format(
            np.sum(
                adata.obs["Tissue"].isin(
                    ["Primary tumour"]
                )
            )
            / adata.n_obs
        ), file=text_file)

In [ ]:
# are any t cell clusters model specific?
sc.pl.umap(adata, color='Model', frameon=False, groups="4T1", save='Tcells_4T1.png')
sc.pl.umap(adata, color='Model', frameon=False, groups="EMT6", save="Tcells_EMT6.png")

In [ ]:
with open(tabdir+"Tcell_4T1_number.txt", "w") as text_file:
    print(
        "Number of Tcells from 4T1: {:.2f}".format(
            np.sum(
                adata.obs["Model"].isin(
                    ["4T1"]
                )
            )
#            / adata.n_obs
        ), file=text_file)

with open(tabdir+"Tcell_4T1_proportion.txt", "w") as text_file:
    print(
        "Proportion of Tcells from 4T1: {:.2f}".format(
            np.sum(
                adata.obs["Model"].isin(
                    ["4T1"]
                )
            )
            / adata.n_obs
        ), file=text_file)

with open(tabdir+"Tcell_EMT6_number.txt", "w") as text_file:
    print(
        "Number of Tcells from EMT6: {:.2f}".format(
            np.sum(
                adata.obs["Model"].isin(
                    ["EMT6"]
                )
            )
#            / adata.n_obs
        ), file=text_file)


with open(tabdir+"Tcell_EMT6_proportion.txt", "w") as text_file:
    print(
        "Proportion of Tcells from EMT6: {:.2f}".format(
            np.sum(
                adata.obs["Model"].isin(
                    ["EMT6"]
                )
            )
            / adata.n_obs
        ), file=text_file)

In [ ]:
# proportionally, what tissue does each cell type come from
tissue_fracs = (
    adata.obs.groupby(["Tissue"])
    .apply(lambda x: x["Tcell_type"].value_counts(normalize=True))
    .unstack()
    .melt(ignore_index=False, var_name="Tcell_type", value_name="fraction")
    .reset_index()
)

tissue_fracs.loc[lambda x: x["fraction"] > 0, "Tissue"].nunique()

ch = (
    alt.Chart(tissue_fracs)
    .mark_bar()
    .encode(
        x="fraction",
        y="Tcell_type",
        color=alt.Color("Tissue", legend=alt.Legend(labelLimit=1e6)),
    )
)

#save(ch, ".png")
#als.save(ch, "Tcells_by_tissue.png")
ch.display()

In [ ]:
# proportionally, what model does each cell type come from
model_fracs = (
    adata.obs.groupby(["Model"])
    .apply(lambda x: x["Tcell_type"].value_counts(normalize=True))
    .unstack()
    .melt(ignore_index=False, var_name="Tcell_type", value_name="fraction")
    .reset_index()
)

model_fracs.loc[lambda x: x["fraction"] > 0, "Model"].nunique()

ch = (
    alt.Chart(model_fracs)
    .mark_bar()
    .encode(
        x="fraction",
        y="Tcell_type",
        color=alt.Color("Model", legend=alt.Legend(labelLimit=1e6)),
    )
)

ch.display()

In [ ]:
# what cell types expand in the t cells with and without immunotherapy

treatment_fracs = (
    adata.obs.groupby(["Treatment"])
    .apply(lambda x: x["Tcell_type"].value_counts(normalize=True))
    .unstack()
    .melt(ignore_index=False, var_name="Tcell_type", value_name="fraction")
    .reset_index()
)

treatment_fracs.loc[lambda x: x["fraction"] > 0, "Treatment"].nunique()

ch = (
    alt.Chart(treatment_fracs)
    .mark_bar()
    .encode(
        x="fraction",
        y="Tcell_type",
        color=alt.Color("Treatment", legend=alt.Legend(labelLimit=1e6)),
    )
)
ch.display()

In [ ]:
sc.pl.umap(adata, color='Treatment', frameon=False, groups="Control IgG", save='Tcells_Control IgG.png')
sc.pl.umap(adata, color='Treatment', frameon=False, groups="PD1+CTLA4", save="Tcells_PD1+CTLA4.png")

In [ ]:
with open(tabdir+"Tcell_IgG_number.txt", "w") as text_file:
    print(
        "Number of Tcells from IgG: {:.2f}".format(
            np.sum(
                adata.obs["Treatment"].isin(
                    ["Control IgG"]
                )
            )
#            / adata.n_obs
        ), file=text_file)

with open(tabdir+"Tcell_IgG_proportion.txt", "w") as text_file:
    print(
        "Proportion of Tcells from IgG: {:.2f}".format(
            np.sum(
                adata.obs["Treatment"].isin(
                    ["Control IgG"]
                )
            )
            / adata.n_obs
        ), file=text_file)

with open(tabdir+"Tcell_PD1+CTLA4_number.txt", "w") as text_file:
    print(
        "Number of Tcells from PD1+CTLA4: {:.2f}".format(
            np.sum(
                adata.obs["Treatment"].isin(
                    ["PD1+CTLA4"]
                )
            )
#            / adata.n_obs
        ), file=text_file)


with open(tabdir+"Tcell_PD1+CTLA4_proportion.txt", "w") as text_file:
    print(
        "Proportion of Tcells from PD1+CTLA4: {:.2f}".format(
            np.sum(
                adata.obs["Treatment"].isin(
                    ["PD1+CTLA4"]
                )
            )
            / adata.n_obs
        ), file=text_file)

In [ ]:
# sc.pl.umap(adata, color=["Tcf7", "Lef1"], vmin=2, vmax=4)
# sc.pl.umap(adata, color=["Tcf7", "Lef1"])

In [ ]:
# sc.pl.heatmap(adata, var_names=["Tcf7", "Lef1"], groupby="Tcell_type", vmin=2.5, vmax=5, figsize=(2, 15))

In [ ]:
## TCR QC
ir.tl.chain_qc(adata)

In [ ]:
ax = ir.pl.group_abundance(adata, groupby="receptor_subtype", target_col="Tissue")
plt.savefig(figdir+"receptor_type_tissue.png", bbox_inches="tight")

In [ ]:
# tell me the number of TRA+TRB chains found: receptor_subtype=TRA+TRB
with open("Fraction of cells with a TCR detected.txt", "w") as text_file:
    print(
        "Fraction of cells with a TCR detected: {:.2f}".format(
            np.sum(
                adata.obs["receptor_subtype"].isin(
                    ["TRA+TRB"]
                )
            )
            / adata.n_obs
        ), file=text_file)
    

with open("Number of cells with a TCR detected.txt", "w") as text_file:
    print(
        "Number of cells with a TCR detected: {:.2f}".format(
            np.sum(
                adata.obs["receptor_subtype"].isin(
                    ["TRA+TRB"]
                )
            )
    #        / adata.n_obs
        ), file=text_file)

In [ ]:
ax = ir.pl.group_abundance(adata, groupby="chain_pairing", target_col="Tissue")
plt.savefig(figdir+"chain_pairing_tissue.png", bbox_inches="tight")

In [ ]:
with open(tabdir+"Fraction of cells with more than one pair of TCRs.txt", "w") as text_file:
    print(
        "Fraction of cells with more than one pair of TCRs: {:.2f}".format(
            np.sum(
                adata.obs["chain_pairing"].isin(
                    ["extra VJ", "extra VDJ", "two full chains"]
                )
            )
            / adata.n_obs
        ), file=text_file)

with open(tabdir+"Number of cells with more than one pair of TCRs.txt", "w") as text_file:
    print(
        "Number of cells with more than one pair of TCRs: {:.2f}".format(
            np.sum(
                adata.obs["chain_pairing"].isin(
                    ["extra VJ", "extra VDJ", "two full chains"]
                )
            )
#            / adata.n_obs
        ), file=text_file)


with open(tabdir+"Number of cells with no IR.txt", "w") as text_file:
    print(
        "Number of cells with no TCRs: {:.2f}".format(
            np.sum(
                adata.obs["chain_pairing"].isin(
                    ["no IR"]
                )
            )
#            / adata.n_obs
        ), file=text_file)

In [ ]:
sc.pl.umap(adata, color="chain_pairing", frameon=False, groups=["no IR", "single pair"])

# ugly colors, need to fix

In [ ]:
l=["no IR", "single pair"]

subset=adata[adata.obs.chain_pairing.isin(l)]



In [ ]:
ax = ir.pl.group_abundance(subset, groupby="Tcell_type", target_col="chain_pairing", figsize=(7, 3))
plt.savefig(figdir+"chain_pairing_Tcells_sp_noIR_inly.png", bbox_inches="tight")

In [ ]:
# even distribution of IR and no IR across cell types. Mostly mt-high and memory_cd4_cd8 have no IR. 
ax = ir.pl.group_abundance(adata, groupby="Tcell_type", target_col="chain_pairing", figsize=(7, 3))
plt.savefig(figdir+"chain_pairing_Tcells.png", bbox_inches="tight")

In [ ]:
# define clonotypes based on sequence identity
ir.pp.ir_dist(adata)
ir.tl.define_clonotypes(adata, receptor_arms="all", dual_ir="primary_only")

In [ ]:
ir.tl.clonal_expansion(adata)
# sc.pl.umap(adata, color=["clonal_expansion", "clone_id_size"], wspace=0.3, frameon=False, save="_clonalexp_size_all.png")
# sc.pl.umap(adata, color=["clonal_expansion"], groups="nan", frameon=False, save="_no_clonal_exp_all.png")

In [ ]:
ir.pl.clonal_expansion(adata, groupby="Tcell_type", clip_at=4, normalize=False, show_nonexpanded=False, figsize=(8, 4))

In [ ]:
ir.pl.clonal_expansion(adata, groupby="Tcell_type", clip_at=4, normalize=False, figsize=(8, 4))
plt.savefig(figdir+"allcell_lonal expansion_all_umap.png", bbox_inches="tight")

In [ ]:
# remove cells with only one clone or no IR
exp=adata[adata.obs.clone_id_size!=1.0]
exp = exp[exp.obs["has_ir"] !="None", :]


In [ ]:
sc.pl.umap(exp, color=["clonal_expansion", "clone_id_size", "Tissue"], size=20, wspace=0.3, save="_clonallyexpanded_cells_tissue.pdf")

In [ ]:
sc.pl.umap(exp, color=["clone_id_size", "clonal_expansion", "Tcell_type" ], size=20, wspace=0.3, save='expanded_clones_umap.png')

In [ ]:
ir.pp.ir_dist(exp)

In [ ]:
ir.tl.define_clonotypes(exp, receptor_arms="all", dual_ir="primary_only")

In [ ]:
ir.tl.clonotype_network(exp, min_cells=4)

In [ ]:
ir.pl.clonotype_network(
    exp, color="Model", base_size=20, label_fontsize=9, panel_size=(7, 7)
)
plt.savefig(figdir+"clonotype_network_nt_all_model.png", bbox_inches="tight")

In [ ]:
# do the above, but with amino acid similarity
ir.pp.ir_dist(
    exp,
    metric="alignment",
    sequence="aa",
    cutoff=10,
)

In [ ]:
ir.tl.define_clonotype_clusters(
    exp, sequence="aa", metric="alignment", receptor_arms="all", dual_ir="any"
)

In [ ]:
ir.tl.clonotype_network(exp, min_cells=3, sequence="aa", metric="alignment")

In [ ]:
ir.pl.clonotype_network(
    exp, color="Model", label_fontsize=9, panel_size=(7, 7), base_size=20)
plt.savefig(figdir+"clonotype_network_aa_all_bymodel.png", bbox_inches="tight")

In [ ]:
ir.pl.clonotype_network(
    exp, color="Tissue", label_fontsize=9, panel_size=(7, 7), base_size=20)
plt.savefig(figdir+"clonotype_network_aa_all_bytissue.png", bbox_inches="tight")

In [ ]:
# #subset out cluster 204 - emt6 cluster. Is this cluster just in one treatmnt group or in both?
# exp.obs.loc[exp.obs["cc_aa_alignment"] == "101", :].groupby(
#     [
#         "receptor_subtype",
#         "Treatment", 
#         "Tissue"
#     ],
#     observed=True,
# ).size().reset_index(name="n_cells")

In [ ]:
ir.pl.clonal_expansion(exp, groupby="Tcell_type", clip_at=4, normalize=False, figsize=(8, 4))
plt.savefig(figdir+"clonal_expansion_byTcell_Type_all.png", bbox_inches="tight")

In [ ]:
ir.pl.clonal_expansion(exp, groupby="Tissue", clip_at=4, normalize=False, figsize=(3,3))
plt.savefig(figdir+"clonal_expansion_byTissue_all.png", bbox_inches="tight")

In [ ]:
ir.pl.clonal_expansion(exp, groupby="Tcell_type", clip_at=4, normalize=True, figsize=(8, 4))
plt.savefig(figdir+"clonal_expansion_byTcell_Type_normalised_all.png", bbox_inches="tight")

In [ ]:
exp

In [ ]:
#### Need to subset the models. How does clonal expansion happen in EMT6 when treated with IT vs control IgG?
l=["EMT6"]
emt6=adata[adata.obs.Model.isin(l)]

In [ ]:
l=["4T1"]
x4t1=adata[adata.obs.Model.isin(l)]

In [ ]:
ir.tl.clonal_expansion(emt6)
# sc.pl.umap(emt6, color=["clonal_expansion", "clone_id_size"], wspace=0.3, save="_emt6_all.png")
# sc.pl.umap(emt6, color=["clonal_expansion"], groups=["nan"], save="_emt6_noIR.png")

In [ ]:
sc.pl.umap(emt6, color=["clone_id_size", "clonal_expansion", "Tcell_type" ], size=10, wspace=0.3, save="_emt6_clonalexpansion_celltype_all")
sc.pl.umap(emt6, color=["clone_id_size", "clonal_expansion", "Tissue" ], size=10, wspace=0.3, save="_emt6_clonalexpansion_tissue_all")

In [ ]:
ir.pl.clonal_expansion(emt6, groupby="Tcell_type", clip_at=4, normalize=False, figsize=(8, 4))
plt.savefig(figdir+"emt6_clonal expansion_all_umap.png", bbox_inches="tight")

In [ ]:
emt6_exp=emt6[emt6.obs.clonal_expansion!="1"]
emt6_exp=emt6_exp[emt6_exp.obs.clone_id_size!=1.0]
emt6_exp = emt6_exp[emt6_exp.obs["has_ir"] !="None", :]

In [ ]:
sc.pl.umap(emt6_exp, color=["clonal_expansion", "Tcell_type" ], size=20, wspace=0.3, save="_emt6_clonalexpansion.png")
sc.pl.umap(emt6_exp, color=["clonal_expansion", "Tissue" ], size=20, wspace=0.3, save="_emt6_clonalexpanion_tissue.png")

In [ ]:
ir.pl.clonal_expansion(emt6_exp, groupby="Tcell_type", clip_at=4, normalize=True, figsize=(8, 4))
plt.savefig(figdir+"emt6_clonalexpanion_tcelltype_normalised.png", bbox_inches="tight")


# ir.pl.clonal_expansion(emt6_exp, groupby="Tcell_type", clip_at=4, normalize=True, figsize=(8, 4))
# plt.savefig(figdir+"emt6_clonalexpanion_tcelltype_normalised.png", bbox_inches="tight")


In [ ]:
ir.tl.clonal_expansion(x4t1)
# sc.pl.umap(x4t1, color=["clonal_expansion", "clone_id_size"], wspace=0.3, save="_x4t1_all.png")
# sc.pl.umap(x4t1, color=["clonal_expansion"], groups="nan", save="x4T1_noIR.png")

In [ ]:
sc.pl.umap(x4t1, color=["clone_id_size", "clonal_expansion", "Tcell_type" ], size=20, wspace=0.3, save="_x4t1_all_clonalexp.png")

In [ ]:
ir.pl.clonal_expansion(x4t1, groupby="Tcell_type", clip_at=4, normalize=False, figsize=(8, 4))
plt.savefig(figdir+"4t1_clonal_expansion_all.png", bbox_inches="tight")

In [ ]:
x4t1_exp=x4t1[x4t1.obs.clonal_expansion!="1"]
x4t1_exp=x4t1_exp[x4t1_exp.obs.clone_id_size!=1.0]
x4t1_exp = x4t1_exp[x4t1_exp.obs["has_ir"] !="None", :]

In [ ]:
sc.pl.umap(x4t1_exp, color=["clone_id_size", "clonal_expansion", "Tcell_type" ], size=20, wspace=0.3,  save="4t1_clonalexpanion.png")
sc.pl.umap(x4t1_exp, color=["clone_id_size", "clonal_expansion", "Tissue" ], size=20, wspace=0.3,  save="4t1_clonalexpansion_tissue.png")

In [ ]:
ir.pl.clonal_expansion(x4t1_exp, groupby="Tcell_type", clip_at=4, normalize=False, figsize=(8, 4))
plt.savefig(figdir+"4t1_clonalexpanion.png", bbox_inches="tight")

# ir.pl.clonal_expansion(x4t1_exp, groupby="Tcell_type", clip_at=4, normalize=True, figsize=(8, 4))
# plt.savefig(figdir+"4t1_clonalexpanion_normalised.png", bbox_inches="tight")

In [ ]:
ir.pl.clonal_expansion(x4t1_exp, groupby="Treatment", clip_at=10, normalize=True)
plt.savefig(figdir+"4t1_clonal_expansion_bytreatment.png", bbox_inches="tight")

In [ ]:
ir.pl.clonal_expansion(emt6_exp, groupby="Treatment", clip_at=10, normalize=True)
plt.savefig(figdir+"emt6_clonal_expansion_bytreatment.png", bbox_inches="tight")    

In [ ]:
l=["PD1+CTLA4"]
emt6_exp_it=emt6_exp[emt6_exp.obs.Treatment.isin(l)]

l=["Control IgG"]
emt6_exp_ig=emt6_exp[emt6_exp.obs.Treatment.isin(l)]

In [ ]:
ir.tl.clonal_expansion(emt6_exp_it)

In [ ]:
emt6_exp_it=emt6_exp_it[emt6_exp_it.obs.clonal_expansion!="1"]
emt6_exp_it=emt6_exp_it[emt6_exp_it.obs.clone_id_size!=1.0]
emt6_exp_it = emt6_exp_it[emt6_exp_it.obs["has_ir"] !="None", :]

In [ ]:
ir.pl.clonal_expansion(emt6_exp_it, groupby="Tcell_type", clip_at=4, normalize=True, figsize=(8, 4))
plt.savefig(figdir+"emt6_it_clonalexpanion_normalised.png", bbox_inches="tight")

In [ ]:
sc.pl.umap(emt6_exp_it, color=["clonal_expansion", "Tcell_type"], wspace=0.3, frameon=False, size=20, save="_emt6_exp_it_umap.png")

In [ ]:
ir.tl.clonal_expansion(emt6_exp_ig)

In [ ]:
emt6_exp_ig=emt6_exp_ig[emt6_exp_ig.obs.clonal_expansion!="1"]
emt6_exp_ig=emt6_exp_ig[emt6_exp_ig.obs.clone_id_size!=1.0]
emt6_exp_ig = emt6_exp_ig[emt6_exp_ig.obs["has_ir"] !="None", :]

In [ ]:
ir.pl.clonal_expansion(emt6_exp_ig, groupby="Tcell_type", clip_at=4, normalize=True, figsize=(8, 4))
plt.savefig(figdir+"emt6_ig_clonalexpanion_normalised.png", bbox_inches="tight")

In [ ]:
ir.pl.clonal_expansion(emt6_exp_ig, groupby="Tissue", clip_at=4, normalize=False, figsize=(3, 3))
plt.savefig(figdir+"emt6_ig_clonalexpanion_tissue.png", bbox_inches="tight")

In [ ]:
ir.pl.clonal_expansion(emt6_exp_it, groupby="Tissue", clip_at=4, normalize=False, figsize=(3, 3))
plt.savefig(figdir+"emt6_it_clonalexpanion_tissue.png", bbox_inches="tight")

In [ ]:
l=["PD1+CTLA4"]
x4t1_exp_it=x4t1_exp[x4t1_exp.obs.Treatment.isin(l)]

l=["Control IgG"]
x4t1_exp_ig=x4t1_exp[x4t1_exp.obs.Treatment.isin(l)]

In [ ]:
ir.tl.clonal_expansion(x4t1_exp_it)

In [ ]:
x4t1_exp_it=x4t1_exp_it[x4t1_exp_it.obs.clonal_expansion!="1"]
x4t1_exp_it=x4t1_exp_it[x4t1_exp_it.obs.clone_id_size!=1.0]
x4t1_exp_it = x4t1_exp_it[x4t1_exp_it.obs["has_ir"] !="None", :]

In [ ]:
ir.pl.clonal_expansion(x4t1_exp_it, groupby="Tcell_type", clip_at=4, normalize=False, figsize=(8, 4))
plt.savefig(figdir+"x4t1_it_clonalexpansion.png", bbox_inches="tight")

In [ ]:
ir.pl.clonal_expansion(x4t1_exp_it, groupby="Tissue", clip_at=4, normalize=False, figsize=(4, 4))
plt.savefig(figdir+"x4t1_it_clonalexpansion_tissue.png", bbox_inches="tight")

In [ ]:
ir.tl.clonal_expansion(x4t1_exp_ig)

In [ ]:
x4t1_exp_ig=x4t1_exp_ig[x4t1_exp_ig.obs.clonal_expansion!="1"]
x4t1_exp_ig=x4t1_exp_ig[x4t1_exp_ig.obs.clone_id_size!=1.0]
x4t1_exp_ig = x4t1_exp_ig[x4t1_exp_ig.obs["has_ir"] !="None", :]

In [ ]:
ir.pl.clonal_expansion(x4t1_exp_ig, groupby="Tcell_type", clip_at=4, normalize=False, figsize=(8, 4))
plt.savefig(figdir+"x4t1_ig_clonalexpansion.png", bbox_inches="tight")

In [ ]:
ir.pl.clonal_expansion(x4t1_exp_ig, groupby="Tissue", clip_at=4, normalize=False, figsize=(4, 4))
plt.savefig(figdir+"x4t1_ig_clonalexpansion_tissue.png", bbox_inches="tight")

In [ ]:
l=["Primary tumour"]
x4t1_exp_ig_pt=x4t1_exp_ig[x4t1_exp_ig.obs.Tissue.isin(l)]

l=["Lymph node"]
x4t1_exp_ig_ln=x4t1_exp_ig[x4t1_exp_ig.obs.Tissue.isin(l)]

In [ ]:
ir.tl.clonal_expansion(x4t1_exp_ig_pt)
ir.tl.clonal_expansion(x4t1_exp_ig_ln)

In [ ]:
x4t1_exp_ig_pt=x4t1_exp_ig_pt[x4t1_exp_ig_pt.obs.clonal_expansion!="1"]
x4t1_exp_ig_pt=x4t1_exp_ig_pt[x4t1_exp_ig_pt.obs.clone_id_size!=1.0]
x4t1_exp_ig_pt = x4t1_exp_ig_pt[x4t1_exp_ig_pt.obs["has_ir"] !="None", :]

x4t1_exp_ig_ln=x4t1_exp_ig_ln[x4t1_exp_ig_ln.obs.clonal_expansion!="1"]
x4t1_exp_ig_ln=x4t1_exp_ig_ln[x4t1_exp_ig_ln.obs.clone_id_size!=1.0]
x4t1_exp_ig_ln = x4t1_exp_ig_ln[x4t1_exp_ig_ln.obs["has_ir"] !="None", :]

In [ ]:
ir.pl.clonal_expansion(x4t1_exp_ig_pt, groupby="Tcell_type", clip_at=4, normalize=False, figsize=(8, 4))
ir.pl.clonal_expansion(x4t1_exp_ig_ln, groupby="Tcell_type", clip_at=4, normalize=False, figsize=(8, 4))

In [ ]:
l=["Primary tumour"]
x4t1_exp_it_pt=x4t1_exp_it[x4t1_exp_it.obs.Tissue.isin(l)]
l=["Lymph node"]
x4t1_exp_it_ln=x4t1_exp_it[x4t1_exp_it.obs.Tissue.isin(l)]


In [ ]:
ir.tl.clonal_expansion(x4t1_exp_it_pt)
ir.tl.clonal_expansion(x4t1_exp_it_ln)

In [ ]:
x4t1_exp_it_pt=x4t1_exp_it_pt[x4t1_exp_it_pt.obs.clonal_expansion!="1"]
x4t1_exp_it_pt=x4t1_exp_it_pt[x4t1_exp_it_pt.obs.clone_id_size!=1.0]
x4t1_exp_it_pt = x4t1_exp_it_pt[x4t1_exp_it_pt.obs["has_ir"] !="None", :]

x4t1_exp_it_ln=x4t1_exp_it_ln[x4t1_exp_it_ln.obs.clonal_expansion!="1"]
x4t1_exp_it_ln=x4t1_exp_it_ln[x4t1_exp_it_ln.obs.clone_id_size!=1.0]
x4t1_exp_it_ln = x4t1_exp_it_ln[x4t1_exp_it_ln.obs["has_ir"] !="None", :]

In [ ]:
ir.pl.clonal_expansion(x4t1_exp_ig_pt, groupby="Tcell_type", clip_at=4, normalize=False, figsize=(5.5, 4))
plt.savefig(figdir+"x4t1_ig_pt_clonalexpansion.png", bbox_inches="tight")

In [ ]:
ir.pl.clonal_expansion(x4t1_exp_it_pt, groupby="Tcell_type", clip_at=4, normalize=False, figsize=(4, 4))
plt.savefig(figdir+"x4t1_it_pt_clonalexpansion.png", bbox_inches="tight")

In [ ]:
ir.pl.clonal_expansion(x4t1_exp_it_ln, groupby="Tcell_type", clip_at=4, normalize=False, figsize=(5, 4))
plt.savefig(figdir+"x4t1_it_ln_clonalexpansion.png", bbox_inches="tight")

In [ ]:
ir.pl.clonal_expansion(x4t1_exp_ig_ln, groupby="Tcell_type", clip_at=4, normalize=False, figsize=(5, 4))
plt.savefig(figdir+"x4t1_ig_ln_clonalexpansion.png", bbox_inches="tight")

In [ ]:
l=["Primary tumour"]
emt6_exp_it_pt=emt6_exp_it[emt6_exp_it.obs.Tissue.isin(l)]
l=["Lymph node"]
emt6_exp_it_ln=emt6_exp_it[emt6_exp_it.obs.Tissue.isin(l)]


In [ ]:
ir.tl.clonal_expansion(emt6_exp_it_pt)
ir.tl.clonal_expansion(emt6_exp_it_ln)

In [ ]:
emt6_exp_it_pt=emt6_exp_it_pt[emt6_exp_it_pt.obs.clonal_expansion!="1"]
emt6_exp_it_pt=emt6_exp_it_pt[emt6_exp_it_pt.obs.clone_id_size!=1.0]
emt6_exp_it_pt = emt6_exp_it_pt[emt6_exp_it_pt.obs["has_ir"] !="None", :]

emt6_exp_it_ln=emt6_exp_it_ln[emt6_exp_it_ln.obs.clonal_expansion!="1"]
emt6_exp_it_ln=emt6_exp_it_ln[emt6_exp_it_ln.obs.clone_id_size!=1.0]
emt6_exp_it_ln = emt6_exp_it_ln[emt6_exp_it_ln.obs["has_ir"] !="None", :]

In [ ]:
ir.pl.clonal_expansion(emt6_exp_it_pt, groupby="Tcell_type", clip_at=4, normalize=False, figsize=(8, 4))
ir.pl.clonal_expansion(emt6_exp_it_ln, groupby="Tcell_type", clip_at=4, normalize=False, figsize=(8, 4))

In [ ]:
ir.tl.clonal_expansion(emt6_exp_it_pt)
ir.tl.clonal_expansion(emt6_exp_it_ln)

In [ ]:

ir.pp.ir_dist(
    emt6_exp,
    metric="alignment",
    sequence="aa",
    cutoff=10,
)

ir.tl.define_clonotype_clusters(
    emt6_exp, sequence="aa", metric="alignment", receptor_arms="all", dual_ir="any"
)

ir.tl.clonotype_network(emt6_exp, min_cells=3, sequence="aa", metric="alignment")

In [ ]:
ir.pl.clonotype_network(
    emt6_exp, color="Tissue", label_fontsize=9, panel_size=(7, 7), base_size=20, frameon=False)
plt.savefig(figdir+"emt6_clone_network_bytissue.png", bbox_inches="tight")

ir.pl.clonotype_network(
    emt6_exp, color="Treatment", label_fontsize=9, panel_size=(7, 7), base_size=20, frameon=False)
plt.savefig(figdir+"emt6_clone_network_bytreatment.png", bbox_inches="tight")

ir.pl.clonotype_network(
    emt6_exp, color="Tcell_type", label_fontsize=9, panel_size=(7, 7), base_size=20, frameon=False)
plt.savefig(figdir+"emt6_clone_network_bycell_type.png", bbox_inches="tight")

Interesting things: Clone 12 is part of a big network found both within the primary tumour and the lymph node, and is there in both control treated and immunotherapy treated mice
Clone 23 is present in both the lymph node and primary tumour, but only when treated with immunotherapy. Evidence new antigens are being presented and activated.

In [ ]:
# # lets isolate clone 12 - what cells is it expressed on?
# emt6_exp.obs.loc[emt6_exp.obs["cc_aa_alignment"] == "12", :].groupby(
#     [
#         "receptor_subtype",
#         "Tcell_type",
#     ],
#     observed=True,
# ).size().reset_index(name="n_cells")

In [ ]:
l=["PD1+CTLA4"]
emt6_exp_it=emt6_exp[emt6_exp.obs.Treatment.isin(l)]

l=["Control IgG"]
emt6_exp_ig=emt6_exp[emt6_exp.obs.Treatment.isin(l)]

In [ ]:
emt6_exp_it=emt6_exp_it[emt6_exp_it.obs.clonal_expansion!="1"]
emt6_exp_it=emt6_exp_it[emt6_exp_it.obs.clone_id_size!=1.0]
emt6_exp_it = emt6_exp_it[emt6_exp_it.obs["has_ir"] !="None", :]

In [ ]:
sc.pl.umap(emt6_exp_it, color=["clonal_expansion", "clone_id_size", "Tcell_type"], size=20, wspace=0.3, save="_clonallyexpanded_cells_EMT6_IT.png")

sc.pl.umap(emt6_exp_ig, color=["clonal_expansion", "clone_id_size", "Tcell_type"], size=20, wspace=0.3, save="_clonallyexpanded_cells_EMT6_ig.png")

In [ ]:
ir.pl.clonal_expansion(emt6_exp_it, groupby="Tcell_type", clip_at=4, normalize=False, figsize=(8, 4))
plt.savefig(figdir+"clonal_expansion_byTcell_Type_emt6_it.png", bbox_inches="tight")

In [ ]:
ir.pl.group_abundance(emt6_exp_it, groupby="cc_aa_alignment", target_col="Tcell_type", max_cols=10)
plt.savefig(figdir+"emt6_IT_clonotype_aa_celltype.png", bbox_inches="tight")

ir.pl.group_abundance(emt6_exp_it, groupby="cc_aa_alignment", target_col="Tcell_type", max_cols=10, normalize=True)
plt.savefig(figdir+"emt6_IT_clonotype_aa_celltype_normalised.png", bbox_inches="tight")

In [ ]:
ir.pl.group_abundance(emt6_exp_it, groupby="clone_id", target_col="Tcell_type", max_cols=10)
plt.savefig(figdir+"emt6_IT_clone_id_celltype.png", bbox_inches="tight")

ir.pl.group_abundance(emt6_exp_it, groupby="clone_id", target_col="Tcell_type", max_cols=10, normalize=True)
plt.savefig(figdir+"emt6_IT_clone_id_celltype_normalised.png", bbox_inches="tight")

In [ ]:
l=["Tfh"]
emt6_exp_it_tfh=emt6_exp_it[emt6_exp_it.obs.Tcell_type.isin(l)]

In [ ]:
sc.pl.umap(emt6_exp_it, color="clone_id", size=20, groups="16000")
sc.pl.umap(emt6_exp_it, color="Tcell_type", size=20)

In [ ]:
ir.pl.group_abundance(emt6_exp_ig, groupby="cc_aa_alignment", target_col="Tcell_type", max_cols=10)
plt.savefig(figdir+"emt6_IgG_clonotype_aa_celltype.png", bbox_inches="tight")

ir.pl.group_abundance(emt6_exp_ig, groupby="cc_aa_alignment", target_col="Tcell_type", max_cols=10, normalize=True)
plt.savefig(figdir+"emt6_IgG_clonotype_aa_celltype_normalised.png", bbox_inches="tight")

In [ ]:
ir.pl.group_abundance(emt6_exp_ig, groupby="clone_id", target_col="Tcell_type", max_cols=10)
plt.savefig(figdir+"emt6_IgG_clonotype_nt_celltype.png", bbox_inches="tight")

ir.pl.group_abundance(emt6_exp_ig, groupby="clone_id", target_col="Tcell_type", max_cols=10, normalize=True)
plt.savefig(figdir+"emt6_IgG_clonotype_nt_celltype_normalised.png", bbox_inches="tight")

In [ ]:
ir.pp.ir_dist(
    x4t1_exp,
    metric="alignment",
    sequence="aa",
    cutoff=10,
)

ir.tl.define_clonotype_clusters(
    x4t1_exp, sequence="aa", metric="alignment", receptor_arms="all", dual_ir="any"
)

ir.tl.clonotype_network(x4t1_exp, min_cells=3, sequence="aa", metric="alignment")

In [ ]:
ir.pl.clonotype_network(
    x4t1_exp, color="Tissue", label_fontsize=9, panel_size=(7, 7), base_size=20, frameon=False)
plt.savefig(figdir+"4t1_clone_network_bytissue.png", bbox_inches="tight")


ir.pl.clonotype_network(
    x4t1_exp, color="Treatment", label_fontsize=9, panel_size=(7, 7), base_size=20, frameon=False)
plt.savefig(figdir+"4t1_clone_network_bytreatment.png", bbox_inches="tight")


ir.pl.clonotype_network(
    x4t1_exp, color="Tcell_type", label_fontsize=9, panel_size=(7, 7), base_size=20, frameon=False)
plt.savefig(figdir+"4t1_clone_network_bycell_type.png", bbox_inches="tight")

In [ ]:
l=["PD1+CTLA4"]
x4t1_exp_it=x4t1_exp[x4t1_exp.obs.Treatment.isin(l)]

l=["Control IgG"]
x4t1_exp_ig=x4t1_exp[x4t1_exp.obs.Treatment.isin(l)]

In [ ]:
ir.pl.group_abundance(x4t1_exp_it, groupby="cc_aa_alignment", target_col="Tcell_type", max_cols=10)
plt.savefig(figdir+"x4t1_IT_clonotype_aa_celltype.png", bbox_inches="tight")

ir.pl.group_abundance(x4t1_exp_it, groupby="cc_aa_alignment", target_col="Tcell_type", max_cols=10, normalize=True)
plt.savefig(figdir+"x4t1_IT_clonotype_aa_celltype_normalised.png", bbox_inches="tight")

In [ ]:
ir.pl.group_abundance(x4t1_exp_it, groupby="clone_id", target_col="Tcell_type", max_cols=10)
plt.savefig(figdir+"x4t1_IT_clone_id_celltype.png", bbox_inches="tight")

ir.pl.group_abundance(x4t1_exp_it, groupby="clone_id", target_col="Tcell_type", max_cols=10, normalize=True)
plt.savefig(figdir+"x4t1_IT_clone_id_celltype_normalised.png", bbox_inches="tight")

In [ ]:
ir.pl.group_abundance(x4t1_exp_ig, groupby="cc_aa_alignment", target_col="Tcell_type", max_cols=10)
plt.savefig(figdir+"4t1_IgG_cc_aa_alignment_celltype.png", bbox_inches="tight")

ir.pl.group_abundance(x4t1_exp_ig, groupby="cc_aa_alignment", target_col="Tcell_type", max_cols=10, normalize=True)
plt.savefig(figdir+"4t1_IgG_cc_aa_alignment_celltype_normalised.png", bbox_inches="tight")

In [ ]:
ir.pl.group_abundance(x4t1_exp_ig, groupby="clone_id", target_col="Tcell_type", max_cols=10)
plt.savefig(figdir+"4t1_IgG_clone_id_celltype.png", bbox_inches="tight")

ir.pl.group_abundance(x4t1_exp_ig, groupby="clone_id", target_col="Tcell_type", max_cols=10, normalize=True)
plt.savefig(figdir+"4t1_IgG_clone_id_celltype_normalised.png", bbox_inches="tight")

where are the tfh cells? are they in the LN or in the primary tumour?

In [ ]:
ir.pl.group_abundance(x4t1_exp_it, groupby="cc_aa_alignment", target_col="Tissue", max_cols=10)
plt.savefig(figdir+"4t1_IT_clonotype_aa_tissue.png", bbox_inches="tight")

ir.pl.group_abundance(x4t1_exp_ig, groupby="cc_aa_alignment", target_col="Tissue", max_cols=10)
plt.savefig(figdir+"4t1_IgG_clonotype_aa_tissue.png", bbox_inches="tight")

In [ ]:
ir.pl.group_abundance(x4t1_exp_it, groupby="clone_id", target_col="Tissue", max_cols=10)
plt.savefig(figdir+"4t1_IT_clonotype_nt_tissue.png", bbox_inches="tight")

ir.pl.group_abundance(x4t1_exp_ig, groupby="clone_id", target_col="Tissue", max_cols=10)
plt.savefig(figdir+"4t1_IgG_clonotype_nt_tissue.png", bbox_inches="tight")

In [ ]:
ir.pl.group_abundance(emt6_exp_it, groupby="cc_aa_alignment", target_col="Tissue", max_cols=10)
plt.savefig(figdir+"emt6_IT_clonotype_aa_tissue.png", bbox_inches="tight")

ir.pl.group_abundance(emt6_exp_ig, groupby="cc_aa_alignment", target_col="Tissue", max_cols=10)
plt.savefig(figdir+"emt6_IgG_clonotype_aa_tissue.png", bbox_inches="tight")

In [ ]:
ir.pl.group_abundance(emt6_exp_it, groupby="clone_id", target_col="Tissue", max_cols=10)
plt.savefig(figdir+"emt6_IT_clonotype_nt_tissue.png", bbox_inches="tight")

ir.pl.group_abundance(emt6_exp_ig, groupby="clone_id", target_col="Tissue", max_cols=10)
plt.savefig(figdir+"emt6_IgG_clonotype_nt_tissue.png", bbox_inches="tight")

In [ ]:
# looks at all expanded cells, separated by location
tissue_fracs = (
    emt6_exp.obs.groupby(["Tissue"])
    .apply(lambda x: x["Tcell_type"].value_counts(normalize=True))
    .unstack()
    .melt(ignore_index=False, var_name="Tcell_type", value_name="fraction")
    .reset_index()
)

tissue_fracs.loc[lambda x: x["fraction"] > 0, "Tissue"].nunique()

ch = (
    alt.Chart(tissue_fracs)
    .mark_bar()
    .encode(
        x="fraction",
        y="Tcell_type",
        color=alt.Color("Tissue", legend=alt.Legend(labelLimit=1e6)),
    )
)

ch.display()

In [ ]:
tissue_fracs = (
    x4t1_exp.obs.groupby(["Tissue"])
    .apply(lambda x: x["Tcell_type"].value_counts(normalize=True))
    .unstack()
    .melt(ignore_index=False, var_name="Tcell_type", value_name="fraction")
    .reset_index()
)

tissue_fracs.loc[lambda x: x["fraction"] > 0, "Tissue"].nunique()

ch = (
    alt.Chart(tissue_fracs)
    .mark_bar()
    .encode(
        x="fraction",
        y="Tcell_type",
        color=alt.Color("Tissue", legend=alt.Legend(labelLimit=1e6)),
    )
)

ch.display()

In [ ]:
#subset just primary tumour for each expanded model and make a network by cell type. Maybe the cd8 cells in the tumour or 4t1 mice are not expanded?

In [ ]:
l=["Primary tumour"]
emt6_exp_pt = emt6_exp[emt6_exp.obs.Tissue.isin(l)]

In [ ]:
ir.pp.ir_dist(
    emt6_exp_pt,
    metric="alignment",
    sequence="aa",
    cutoff=10,
)

ir.tl.define_clonotype_clusters(
    emt6_exp_pt, sequence="aa", metric="alignment", receptor_arms="all", dual_ir="any"
)

ir.tl.clonotype_network(emt6_exp_pt, min_cells=3, sequence="aa", metric="alignment")

In [ ]:
ir.pl.clonotype_network(
    emt6_exp_pt, color="Tissue", label_fontsize=9, panel_size=(7, 7), base_size=20, frameon=False)

ir.pl.clonotype_network(
    emt6_exp_pt, color="Treatment", label_fontsize=9, panel_size=(7, 7), base_size=20, frameon=False)

ir.pl.clonotype_network(
    emt6_exp_pt, color="Tcell_type", label_fontsize=9, panel_size=(7, 7), base_size=20, frameon=False)

The top plot shows that most of the expanded clones comes from immunotherapy treated clones. 
These clones are expressed on many cells including TFh, T effector 

In [ ]:
l=["Primary tumour"]
x4t1_exp_pt = x4t1_exp[x4t1_exp.obs.Tissue.isin(l)]

In [ ]:
ir.pp.ir_dist(
    x4t1_exp_pt,
    metric="alignment",
    sequence="aa",
    cutoff=10,
)

ir.tl.define_clonotype_clusters(
    x4t1_exp_pt, sequence="aa", metric="alignment", receptor_arms="all", dual_ir="any"
)

ir.tl.clonotype_network(x4t1_exp_pt, min_cells=3, sequence="aa", metric="alignment")

In [ ]:
ir.pl.clonotype_network(
    x4t1_exp_pt, color="Tissue", label_fontsize=9, panel_size=(7, 7), base_size=20, frameon=False)

ir.pl.clonotype_network(
    x4t1_exp_pt, color="Treatment", label_fontsize=9, panel_size=(7, 7), base_size=20, frameon=False)

ir.pl.clonotype_network(
    x4t1_exp_pt, color="Tcell_type", label_fontsize=9, panel_size=(7, 7), base_size=20, frameon=False)

Expanded clones in both immunotherapy treated and control treated 4t1 mice.
In control treated tumours, what cells are clonally expanded?
#41, 45, 97, 89, 64, 49, 57

In [ ]:
# lets see where the clonal expansion is in the primary tumour cells from 4t1 and emt6
sc.pl.umap(x4t1_exp_pt, color=["clone_id_size", "clonal_expansion", "Tcell_type"], size=20, wspace=0.3)
sc.pl.umap(emt6_exp_pt, color=["clone_id_size", "clonal_expansion", "Tcell_type"], size=20, wspace=0.3)

could be as simple as the relative absense of clonally expanded tregs and tfhs in emt6 tumours. But also less T cells in emt6s
4t1= 398 clonally expanded intratumoural t cells
emt6 = 180 clonally expended intratumoural t cells

can i make a proportion graph within each of these models, normalised by total cell number?

In [ ]:

tissue_fracs = (
    emt6_exp_pt.obs.groupby(["Treatment"])
    .apply(lambda x: x["Tcell_type"].value_counts(normalize=True))
    .unstack()
    .melt(ignore_index=False, var_name="Tcell_type", value_name="fraction")
    .reset_index()
)

tissue_fracs.loc[lambda x: x["fraction"] > 0, "Treatment"].nunique()

ch = (
    alt.Chart(tissue_fracs)
    .mark_bar()
    .encode(
        x="fraction",
        y="Tcell_type",
        color=alt.Color("Treatment", legend=alt.Legend(labelLimit=1e6)),
    )
)

ch.display()

In [ ]:

tissue_fracs = (
    x4t1_exp_pt.obs.groupby(["Treatment"])
    .apply(lambda x: x["Tcell_type"].value_counts(normalize=True))
    .unstack()
    .melt(ignore_index=False, var_name="Tcell_type", value_name="fraction")
    .reset_index()
)

tissue_fracs.loc[lambda x: x["fraction"] > 0, "Treatment"].nunique()

ch = (
    alt.Chart(tissue_fracs)
    .mark_bar()
    .encode(
        x="fraction",
        y="Tcell_type",
        color=alt.Color("Treatment", legend=alt.Legend(labelLimit=1e6)),
    )
)

ch.display()

In [ ]:
# # how does entropy change between treatment groups
# ax = ir.pl.alpha_diversity(
#     emt6_exp_ig, metric="normalized_shannon_entropy", groupby="Tcell_type", figsize=(8, 4))
# #plt.savefig(figdir+"emt6_shannondiveristy_Tcelltype.png", bbox_inches="tight")

In [ ]:
# ax = ir.pl.alpha_diversity(
#     emt6_exp_it, metric="normalized_shannon_entropy", groupby="Tcell_type", figsize=(8, 4))

In [ ]:
# ax = ir.pl.alpha_diversity(
#     x4t1_exp_ig, metric="normalized_shannon_entropy", groupby="Tcell_type", figsize=(8, 4))

In [ ]:
# ax = ir.pl.alpha_diversity(
#     x4t1_exp_it, metric="normalized_shannon_entropy", groupby="Tcell_type", figsize=(8, 4))

Is the clone 16000 truely found on  tfh or is it a cd8 positive cell that looks like a tfh?

In [ ]:
sc.pl.umap(emt6_exp_it, color="clone_id", groups="16000", size=25, frameon=False, save="_16000_clone_id.png")
sc.pl.umap(emt6_exp_it, color=["Cd8a", "Cd4"], size=25, frameon=False, save="_16000_CD8_CD8.png")
sc.pl.umap(emt6_exp_it, color=["Tcell_type"], size=25, frameon=False, save="_16000_Tcelltype.png")

In [ ]:
sc.pl.umap(emt6_exp_it, color=["Cd274", "Cd40lg", "Icos", "Cxcr3"], size=20, frameon=False, save="_16000_tfh_markers.png")

In [ ]:
# or is it easier if I subset clone 16000 and do it that way?
l=["16000"]
x16=emt6_exp_it[emt6_exp_it.obs.clone_id.isin(l)]

In [ ]:
sc.pl.umap(x16, color="Tcell_type", size=70, save="_x16_tcelltype.png")

In [ ]:
sc.pl.umap(x16, color="clone_id", size=70, save="_x16.png")

In [ ]:
sc.pl.umap(x16, color=["Cd8a", "Cd4"], size=70, vmin=0, vmax=3, save="_x16_cd8cd4.png")

In [ ]:
sc.pl.umap(x16, color=["Cd274", "Cd40lg", "Icos", "Cxcr3"], size=20, save="_x16_tfh_markers.png")

In [ ]:
sc.pl.umap(emt6_exp_it, color=["Cd8a", "Cd4", "Tcell_type"], size=20, save="_emt6_exp_it_cd8cd4tcelltype.png")

In [ ]:
x16

In the 4t1 mice, is there truly a shared receptor between tfh and treg?

In [ ]:
sc.pl.umap(x4t1_exp_it, color="clone_id", groups="390", size=30)

In [ ]:
l=["390"]
x390=x4t1_exp_it[x4t1_exp_it.obs.clone_id.isin(l)]

In [ ]:
sc.pl.umap(x390, color=["Tcell_type", "Cd40lg", "Foxp3"], size=30)